# Worksheet 07

Name:  Isabella Taboada 

UID: U52535262

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.5045606404655936, 5.773890042333923, 3.3968450235931655, 3.28434038780643, 5.490848786180199, 3.948623735613732, 5.532154149115366, 4.128485290160749, 5.085254588992182, 5.48503445927929]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean = 8
stdev = 1

c2 = np.random.normal(mean, stdev, 10).tolist()
print(c2)

[8.568676810586657, 7.6681013382582135, 9.211109141267373, 8.68018591459764, 7.197905140910839, 8.465951522731926, 7.166956010115087, 7.864429146961898, 5.939649812457912, 7.591850617554125]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.591850617554125, 5.48503445927929, 5.939649812457912, 7.864429146961898, 5.085254588992182, 7.166956010115087, 8.465951522731926, 4.128485290160749, 7.197905140910839, 5.532154149115366]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

For c1: mean = 5 and stdev = 1 
For c2: mean = 8 and stdev = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1)/len(c1), sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.591850617554125, 7.864429146961898, 7.166956010115087, 8.465951522731926, 7.197905140910839]
[5.48503445927929, 5.939649812457912, 5.085254588992182, 4.128485290160749, 5.532154149115366]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 7.657418487654775,  mean_2 = 5.2341156600011
var_1 = 0.2305168370355112,  var_2 = 0.37882875165658153


/Users/isa/Library/Python/3.8/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


The estimated values are pretty close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  7.591850617554125
probability of observing that point if it came from cluster 0 =  1.6620307562569363
probability of observing that point if it came from cluster 1 =  4.085623174135405e-09
point =  5.48503445927929
probability of observing that point if it came from cluster 0 =  8.976215112903034e-20
probability of observing that point if it came from cluster 1 =  0.845671721534426
point =  5.939649812457912
probability of observing that point if it came from cluster 0 =  1.5139851833823946e-12
probability of observing that point if it came from cluster 1 =  0.1858990435117845
point =  7.864429146961898
probability of observing that point if it came from cluster 0 =  1.1563463823525324
probability of observing that point if it came from cluster 1 =  3.580888914823784e-11
point =  5.085254588992182
probability of observing that point if it came from cluster 0 =  1.5920902498743022e-27
probability of observing that point if it came from cluster 1 =  0.9748490011791384
point =  7

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.4999980495757582,  P(C_2) = 0.5000019504242417
mean_1 = 7.657420360527279,  mean_2 = 5.234123240043493
var_1 = 0.23051683613395962,  var_2 = 0.3788420041934474


The values are pretty similar to the original ones we got from K means

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


7.591850617554125
Probability of coming from C_1 = 0.99999999753821
Probability of coming from C_2 = 2.461789951939708e-09

5.48503445927929
Probability of coming from C_1 = 1.0613470713910808e-19
Probability of coming from C_2 = 1.0

5.939649812457912
Probability of coming from C_1 = 8.142560799729226e-12
Probability of coming from C_2 = 0.9999999999918574

7.864429146961898
Probability of coming from C_1 = 0.9999999999689771
Probability of coming from C_2 = 3.102277610002817e-11

5.085254588992182
Probability of coming from C_1 = 1.6330655717117362e-27
Probability of coming from C_2 = 1.0

7.166956010115087
Probability of coming from C_1 = 0.999986969914103
Probability of coming from C_2 = 1.3030085897101148e-05

8.465951522731926
Probability of coming from C_1 = 0.999999999999955
Probability of coming from C_2 = 4.4962374484528926e-14

4.128485290160749
Probability of coming from C_1 = 1.4956707586287229e-49
Probability of coming from C_2 = 1.0

7.197905140910839
Probability of comi

The probabilities here are much higher/ more definitive.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[0, 1, 1, 0, 1, 0, 0, 1, 0, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.